# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [15]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import gmaps.datasets
from pprint import pprint 

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [16]:
gmaps.configure(api_key=g_key)


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [17]:
csv_path = "../output_data/cities.csv"
# Import the books.csv file as a DataFrame
vac_df = pd.read_csv(csv_path, encoding="utf-8")


In [18]:
locations = vac_df[['lat', 'lng']]
weights = vac_df['Humidity']
fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights))
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [19]:
fav_vac= vac_df.loc[(vac_df["Maximum Temperature"]<=30) & (vac_df["Maximum Temperature"]>=5)]
fav_vac= fav_vac.loc[(vac_df["Wind Speed"]<=10)]
fav_vac= fav_vac.loc[(vac_df["Cloudiness"]==0)]
fav_vac


,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,lat,lng,Maximum Temperature,Wind Speed
13,14,airai,0,TL,1592706123,63,-8.93,125.41,20.85,2.05
14,15,komsomolskiy,0,UZ,1592706239,22,40.43,71.72,25.00,1.00
15,16,tiksi,0,RU,1592706239,34,71.69,128.87,20.27,2.45
17,18,vilhena,0,BR,1592706025,72,-12.74,-60.15,20.29,1.45
26,27,saldanha,0,ZA,1592705967,93,-33.01,17.94,14.00,4.10
...,...,...,...,...,...,...,...,...,...,...
484,498,yulara,0,AU,1592706055,35,-25.24,130.99,14.00,3.10
490,504,port arthur,0,CN,1592706300,51,38.80,121.27,28.00,4.00
492,506,ushtobe,0,KZ,1592706301,26,45.25,77.98,19.40,3.69
507,522,yatou,0,CN,1592706303,57,37.15,122.38,25.47,5.38


In [20]:
#A max temperature lower than 80 degrees but higher than 70.
#Wind speed less than 10 mph.
#Zero cloudiness.
# Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [21]:
hotel_df=fav_vac
hotel_df


,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,lat,lng,Maximum Temperature,Wind Speed
13,14,airai,0,TL,1592706123,63,-8.93,125.41,20.85,2.05
14,15,komsomolskiy,0,UZ,1592706239,22,40.43,71.72,25.00,1.00
15,16,tiksi,0,RU,1592706239,34,71.69,128.87,20.27,2.45
17,18,vilhena,0,BR,1592706025,72,-12.74,-60.15,20.29,1.45
26,27,saldanha,0,ZA,1592705967,93,-33.01,17.94,14.00,4.10
...,...,...,...,...,...,...,...,...,...,...
484,498,yulara,0,AU,1592706055,35,-25.24,130.99,14.00,3.10
490,504,port arthur,0,CN,1592706300,51,38.80,121.27,28.00,4.00
492,506,ushtobe,0,KZ,1592706301,26,45.25,77.98,19.40,3.69
507,522,yatou,0,CN,1592706303,57,37.15,122.38,25.47,5.38


In [22]:
hotel_df["Hotel Name"]=''
hotel_df.reset_index(drop=True, inplace=True)
del hotel_df['Unnamed: 0']
hotel_df

,City,Cloudiness,Country,Date,Humidity,lat,lng,Maximum Temperature,Wind Speed,Hotel Name
0,airai,0,TL,1592706123,63,-8.93,125.41,20.85,2.05,
1,komsomolskiy,0,UZ,1592706239,22,40.43,71.72,25.00,1.00,
2,tiksi,0,RU,1592706239,34,71.69,128.87,20.27,2.45,
3,vilhena,0,BR,1592706025,72,-12.74,-60.15,20.29,1.45,
4,saldanha,0,ZA,1592705967,93,-33.01,17.94,14.00,4.10,
...,...,...,...,...,...,...,...,...,...,...
90,yulara,0,AU,1592706055,35,-25.24,130.99,14.00,3.10,
91,port arthur,0,CN,1592706300,51,38.80,121.27,28.00,4.00,
92,ushtobe,0,KZ,1592706301,26,45.25,77.98,19.40,3.69,
93,yatou,0,CN,1592706303,57,37.15,122.38,25.47,5.38,


In [23]:
#Documentation for google map hotels

b_url="https://maps.googleapis.com/maps/api/place/nearbysearch/json"  ##url to send the request
hotel_parameters={
    "radius": 5000,
    "type": "Hotel",
    "keyword":"Hostel",
    "key": g_key}
for i in range(0, len(hotel_df)):
    city = hotel_df.iloc[i,0]
    hotel_parameters['location']=f'{hotel_df.iloc[i,5]}, {hotel_df.iloc[i,6]}'
    data=requests.get(b_url, hotel_parameters).json()
    
    try:
        hotel_df.iloc[i,9]=data['results'][0]['name']
        print(f" The Closest Hotel in {city} is {data['results'][0]['name']}.")
        
    except (KeyError, IndexError):
        print("There is no Hotel in the vicinity... skipping.")
        
hotel_df

There is no Hotel in the vicinity... skipping.
There is no Hotel in the vicinity... skipping.
 The Closest Hotel in tiksi is Arktika.
 The Closest Hotel in vilhena is Hotel Santa Rosa Palace.
 The Closest Hotel in saldanha is Saldanha Holiday Resort.
 The Closest Hotel in cidreira is Pond Country Club.
 The Closest Hotel in arraial do cabo is Enseada Hostel.
 The Closest Hotel in atuona is CHALETS HANAKEE.
 The Closest Hotel in port elizabeth is B's Place.
 The Closest Hotel in busselton is Busselton Backpackers.
There is no Hotel in the vicinity... skipping.
 The Closest Hotel in castro is Hotel Hospedaria Central.
 The Closest Hotel in port alfred is Port Alfred Backpackers.
 The Closest Hotel in constantine is Hotel ELBEY Constantine.
 The Closest Hotel in bredasdorp is Albert Myburgh Sec Sch Hostel.
 The Closest Hotel in itarema is COCO-KNOTS Kite Lodge & Hostel.
 The Closest Hotel in east london is Sugarshack Backpackers.
 The Closest Hotel in nizhnyaya poyma is Gostinitsa "Medvez

,City,Cloudiness,Country,Date,Humidity,lat,lng,Maximum Temperature,Wind Speed,Hotel Name
0,airai,0,TL,1592706123,63,-8.93,125.41,20.85,2.05,
1,komsomolskiy,0,UZ,1592706239,22,40.43,71.72,25.00,1.00,
2,tiksi,0,RU,1592706239,34,71.69,128.87,20.27,2.45,Arktika
3,vilhena,0,BR,1592706025,72,-12.74,-60.15,20.29,1.45,Hotel Santa Rosa Palace
4,saldanha,0,ZA,1592705967,93,-33.01,17.94,14.00,4.10,Saldanha Holiday Resort
...,...,...,...,...,...,...,...,...,...,...
90,yulara,0,AU,1592706055,35,-25.24,130.99,14.00,3.10,"sydney central yha, Ayers Rock - Outback Pione..."
91,port arthur,0,CN,1592706300,51,38.80,121.27,28.00,4.00,Jinhai Bay Hostel
92,ushtobe,0,KZ,1592706301,26,45.25,77.98,19.40,3.69,
93,yatou,0,CN,1592706303,57,37.15,122.38,25.47,5.38,Huaning Hostel


In [24]:
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
hotel_df.dropna(subset=['Hotel Name'], inplace=True)
hotel_df

,City,Cloudiness,Country,Date,Humidity,lat,lng,Maximum Temperature,Wind Speed,Hotel Name
2,tiksi,0,RU,1592706239,34,71.69,128.87,20.27,2.45,Arktika
3,vilhena,0,BR,1592706025,72,-12.74,-60.15,20.29,1.45,Hotel Santa Rosa Palace
4,saldanha,0,ZA,1592705967,93,-33.01,17.94,14.00,4.10,Saldanha Holiday Resort
5,cidreira,0,BR,1592706241,85,-30.18,-50.21,18.99,4.82,Pond Country Club
6,arraial do cabo,0,BR,1592706241,87,-22.97,-42.03,23.22,5.55,Enseada Hostel
...,...,...,...,...,...,...,...,...,...,...
89,kautokeino,0,NO,1592706299,63,69.01,23.04,13.44,4.44,Thon Hotel Kautokeino
90,yulara,0,AU,1592706055,35,-25.24,130.99,14.00,3.10,"sydney central yha, Ayers Rock - Outback Pione..."
91,port arthur,0,CN,1592706300,51,38.80,121.27,28.00,4.00,Jinhai Bay Hostel
93,yatou,0,CN,1592706303,57,37.15,122.38,25.47,5.38,Huaning Hostel


In [25]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>city</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["lat", "lng"]]

In [26]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations,info_box_content=hotel_info)

# Display figure
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))

In [14]:
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
hotel_df.dropna(subset=['Hotel Name'], inplace=True)
hotel_df.drop(hotel_df.columns[[1, 3,4,5,6,7,8]], axis = 1, inplace = True) 
hotel_df

,City,Country,Hotel Name
0,busselton,AU,Fairlawn Estate
1,port hedland,AU,Bruno's Ocean Lodge
2,picos,BR,Pousada Maravilha
3,east london,ZA,Sugarshack Backpackers
4,sitio novo do tocantins,BR,Borges Hotel
5,lebu,CL,Hostal Mackay Lebu
6,atuona,PF,CHALETS HANAKEE
7,carnarvon,AU,Backpackers Inn
8,geraldton,AU,Geraldton Backpackers on the Foreshore
